# Group Bar Chart 

This notebook has the notes for the for the ```gen_grouped_bar()``` function. 

Original code is from https://bokeh.pydata.org/en/latest/docs/gallery/bar_pandas_groupby_nested.html 

In [1]:
import pandas as pd

/home/lightner/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/lightner/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
import pandas as pd
import matplotlib
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.models import ColumnDataSource, Div, HoverTool, LabelSet
from bokeh.models.glyphs import Ellipse

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)


/home/lightner/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/lightner/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## Generate practice dataset 

In [4]:


text = ['Health', 'Education', 'Gender', 'Macro', 'Finance', 'PSD', 'Conflict', 'Taxation', 'Democracy']
# example factor:
f1 = np.array([100, 140, 350, 400, 203, 504, 605, 304, 405])
f2 = np.array([60, 530, 540, 330, 200, 406, 709, 403, 509])
f3 = np.array([700, 400, 400, 600 , 320, 670, 302, 609, 304])


# data frame 
df = pd.DataFrame({'text': text, 'f1': f1, 'f2': f2, 'f3': f3})

df.head()

,text,f1,f2,f3
0,Health,100,60,700
1,Education,140,530,400
2,Gender,350,540,400
3,Macro,400,330,600
4,Finance,203,200,320


In [15]:
def wide_to_long(df, cat_var, value_vars):
    ''' generate the dataset in ESDB long format.'''
    # reorder the dataframe is necessary for the rename code to work properly 
    # the wide to long function needs a common begining to each column to be shifted
    
    
    df = df[[cat_var] + value_vars]
    
    # rename columns 
    df.columns = [cat_var] + ['f'+str(f) for f in range(0, len(df.columns[1:]))]
    
    # move from wide to long format
    l = pd.wide_to_long(df, stubnames='f', i=[cat_var], j='var').reset_index()  
    
    return l

df_plot = long(df, 'text', ['f1', 'f2', 'f3'])
df_plot.head()

,text,var,f
0,Health,0,100
1,Education,0,140
2,Gender,0,350
3,Macro,0,400
4,Finance,0,203


## Generate Bokeh Visual - Bar chart 

In [127]:
# new packages for the pyADIVSE package 
from bokeh.transform import factor_cmap

def new_style(p): 

    
    #######################
    ###  title 
    #######################
    p.title.text_color = '#383951'
    p.title.text_font = "Work Sans"
    p.title.text_font_style = "bold"    
    p.title.text_font_size= '15pt' 
    
    #######################
    ### Grid options 
    #######################
    
    # grid lines 
    p.grid.grid_line_color='white'
    p.grid.grid_line_width=1.5
    p.grid.grid_line_alpha=0.2
    
    
    
    #######################
    #### Axis styles 
    #######################
    
    ### labels
    p.xaxis.axis_label = ''
    p.yaxis.axis_label = ''

    
    p.axis.axis_label_text_color = '#999999'
    p.axis.axis_label_text_font_style = 'normal'
    p.axis.axis_label_text_font = "Work Sans"
    p.axis.major_label_text_font_size = '8pt'
    p.axis.major_label_text_font = "Work Sans"
    p.axis.axis_label_text_font_size = '10pt'    
    
    ## axis colors 
    #p.axis.line_color = ''
    #p.yaxis.axis_label = ''
    
    # axis  line options 
    p.axis.axis_line_color = '#999999'
    p.axis.axis_line_alpha = .8
    p.axis.axis_line_width = 2

    
    
    ### axis visable
    #p.yaxis.visible = False  
    #p.xaxis.visible = False
    #p.axis.axis_line_color=None
    
    
    ### major ticks
    p.axis.major_label_text_color='#999999'
    p.axis.major_tick_line_color='#999999'
    
    ### minor ticks 
    p.xaxis.minor_tick_line_color = None # turn off x-axis minor ticks
    p.yaxis.minor_tick_line_color = None
    
    
    ######################
    #### Legend styles
    ######################
    
    p.legend.label_text_font =  "Work Sans"
    p.legend.label_text_font_size = '11pt'
    p.legend.background_fill_alpha = 0.3

    

    ####################
    # plot outline
    ####################
    
    p.outline_line_color = None
    
    
    return p


def gen_grouped_bar(data, cat_var_member, cat_var_group, value_var ='value_start', 
                    cat_var_member_name='series_name', cat_var_group_name='country_id', 
                   title_text='Grouped Bar - Example'): 
    ''' this function generated a group bar chart where the the cat_var_member data are 
    nested under the group variables, colored by group variables. Can change which varaibles refer to 
    the group var and the member by changing the (*name) varialbes'''
    
    
    ########################
    #### prep data 
    ########################
    
    # select the data of interest
    data = data[data[cat_var_member_name].isin(cat_var_member) &
               data[cat_var_group_name].isin(cat_var_group)]
    
    
    # make sure cat_vars are strings types (object)
    data[cat_var_member_name] = data[cat_var_member_name].astype(str)
    data[cat_var_group_name] = data[cat_var_group_name].astype(str)

    # generate groupby 
    group = data.groupby((cat_var_group_name, cat_var_member_name))

    ###### generate palette 
    palette = {'USAID Blue': '#002F6C', 'USAID Red': '#BA0C2F', 'Rich Black': '#212721', 'Medium Blue': '#0067B9',
    'Light Blue': '#A7C6ED', 'Dark Red': '#651D32', 'Dark Gray': '#6C6463', 'Medium Gray': '#8C8985', 'Light Gray': '#CFCDC9'}
    palette = [palette[i] for i in list(palette.keys())[0:len(category_selection)]]
    
    
    # generate factor_cmap based on the group member (the name can be anything - just referable)
    name = cat_var_group_name + '_'+ cat_var_member_name
    index_cmap = factor_cmap(name, palette=palette, factors=sorted(data[cat_var_group_name].unique()), end=1)
    
    name_tip= '@'+name
    value_tip = '@'+value_var+'_mean{0.0}'
    ##########################
    ##### Generate plot 
    ##########################
    
    
    # generate figure 
    p = figure(plot_width=800, plot_height=300, title="Mean MPG by # Cylinders and Manufacturer",
           x_range=group, toolbar_location=None, tooltips=[('Category: ', name_tip), 
                                                           ('Value: ', value_tip)])
    
    
    # generate bar graph
    p.vbar(x=name, top=value_var+'_mean', width=.8, source=group,
           color=index_cmap, line_width=3,fill_alpha =.6)
    

    
    p.y_range.start = 0
    p.x_range.range_padding = 0.05
    p.xgrid.grid_line_color = None
    #p.xaxis.axis_label = "Manufacturer grouped by # Cylinders"
    p.xaxis.major_label_orientation = 1.2
    p.outline_line_color = None
    
    p = new_style(p)
    
    ## title 
    p.title.text_font_size= '15pt'
    p.title.text = title_text
    
    
    return p
    
    

### Trial 

In [21]:
# selection of categories 
category_selection = ['Conflict', 'Democracy', 'Education', 'Finance', 'Gender',
       'Health', 'Macro', 'PSD', 'Taxation']
# selection country 
country_selection = [0,1,2]



In [128]:
# generatae the plot
p = gen_grouped_bar(df_plot, cat_var_member=category_selection, 
               cat_var_group=country_selection)

In [129]:
# show the plot
show(p)